In [1]:
!pip install transformers
!pip install pandas
!pip install torch torchvision
!pip install sentencepiece
!pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.5 MB/s eta 0:00:00


In [2]:
# Importing the required libraries
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import torch

In [3]:
# Function to run inference for FAQ generation
def generate_faq(model_path, inference_data_path, text_column, output_file):
    # Load the saved model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    tokenizer = T5Tokenizer.from_pretrained("T5-base")

    # Load the inference dataset
    inference_df = pd.read_csv(inference_data_path)
    generated_faqs = []

    # Generate FAQ for each summarized text
    for index, row in inference_df.iterrows():
        input_text = row[text_column]

        # Tokenize the input text and generate an FAQ ID
        inputs = tokenizer("generate: " + input_text, return_tensors="pt", max_length=512, truncation=True)
        with torch.no_grad():  # disable gradient calculation to save memory
            faq_ids = model.generate(inputs.input_ids, num_beams=4, min_length=30, max_length=100, early_stopping=True)

        # Decode the FAQ ID and append to the list of generated FAQs
        generated_faq = tokenizer.decode(faq_ids[0], skip_special_tokens=True)
        generated_faqs.append(generated_faq)

    # Save the generated FAQs to a CSV file
    output_df = pd.DataFrame({
        text_column: inference_df[text_column],
        "Generated_FAQ": generated_faqs
    })
    output_df.to_csv(output_file, index=False)

In [4]:
# Define the paths and column names
model_path = "/content/drive/MyDrive/NLP_Data/Faq/faq_model"  # Replace this with your model path
inference_data_path = "/content/test.csv"  # Replace this with your inference data path
text_column = "summarized_text"
output_file = "/content/generated_faqs.csv"  # Replace this with your desired output file name

In [5]:
# Run the inference
generate_faq(model_path, inference_data_path, text_column, output_file)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
